# 走行

このNotebookで自動走行をおこないます。

## Jetsonの認識

In [ ]:
import Jetson.GPIO as GPIO

BOARD_NAME=GPIO.gpio_pin_data.get_data()[0]
if BOARD_NAME == "JETSON_NX":
    print("Jetson Xavier NXを認識")
    I2C_BUSNUM = 8
    MODE = 2
elif BOARD_NAME == "JETSON_XAVIER":
    print("Jetson AGX Xavierを認識")
    I2C_BUSNUM = 8
    MODE = 2
elif BOARD_NAME == "JETSON_NANO":
    print("Jetson Nanoを認識")
    I2C_BUSNUM = 1
    MODE = 0
elif BOARD_NAME == "JETSON_ORIN":
    print("Jetson AGX Orinを認識")
    I2C_BUSNUM = 7
    MODE = 0
elif BOARD_NAME == "JETSON_ORIN_NANO":
    print("Jetson Orin Nanoを認識")
    I2C_BUSNUM = 7
    MODE = 0

## ログの表示用 Widget

In [ ]:
import ipywidgets
from ipywidgets import Button, Layout, Textarea, HBox, VBox, Label
import os
import glob

l = Layout(flex='0 1 auto', height='100px', min_height='100px', width='auto')
process_widget = ipywidgets.Textarea(description='ログ', value='', layout=l)

process_no = 0
def write_log(msg):
    global process_widget, process_no
    process_no = process_no + 1
    process_widget.value = str(process_no) + ": " + msg + "\n" + process_widget.value

## PWMの値の読み込み

In [ ]:
import Fabo_PCA9685
import time
import pkg_resources
import smbus
import time
import json

SMBUS='smbus'
BUSNUM=I2C_BUSNUM
SERVO_HZ=60
INITIAL_VALUE=300
bus = smbus.SMBus(BUSNUM)
PCA9685 = Fabo_PCA9685.PCA9685(bus,INITIAL_VALUE,address=0x40)
PCA9685.set_hz(SERVO_HZ)

STEERING_CH = 0
THROTTLE_CH = 1
direction = 0

pwm_front = 0
pwm_back = 0

with open('pwm_params.json') as f:
    json_str = json.load(f)
    
    pwm_stop = json_str["pwm_speed"]["stop"]
    pwm_front = json_str["pwm_speed"]["front"]
    pwm_back = json_str["pwm_speed"]["back"]
    pwm_left = json_str["pwm_steering"]["left"]
    pwm_center = json_str["pwm_steering"]["center"]
    pwm_right = json_str["pwm_steering"]["right"]

    
if pwm_front >= pwm_back:
    direction = 0
else:
    direction = 1
    
PCA9685.set_channel_value(STEERING_CH, pwm_center)
PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)

## カメラの読込

この部分でエラーが発生する場合は、Jetsonの再起動をお願いします。<br>
それでも、カメラが認識できない場合は、ケーブルの接続確認をしてください。


In [ ]:
from jetcam.csi_camera import CSICamera

camera = CSICamera(width=224, height=224, capture_fps=30)

## 走行処理

In [ ]:
import torch
import threading
from utils import preprocess
import subprocess
import cv2
import time
from torch2trt import TRTModule
import subprocess
import datetime

IMG_WIDTH=224

model_widget = ipywidgets.Dropdown(options=[],description='モデル')
model_time_widget = ipywidgets.Label(description='作成日時')
load_button = ipywidgets.Button(description='Load')
run_button = ipywidgets.Button(description='Run')
stop_button = ipywidgets.Button(description='Stop')
pwm_left_widget = ipywidgets.IntText(value=pwm_left,description='PWM 左')
pwm_center_widget = ipywidgets.IntText(value=pwm_center,description='PWM 中央')
pwm_right_widget = ipywidgets.IntText(value=pwm_right,description='PWM 右')
check_left_button = ipywidgets.Button(description='チェック左')
check_center_button = ipywidgets.Button(description='チェック中央')
check_right_button = ipywidgets.Button(description='チェック右')
pwm_stop_widget = ipywidgets.IntText(value=pwm_stop,description='PWM 停止')
name_widget = ipywidgets.Text(description='映像の保存先')
record_box = ipywidgets.Checkbox(False, description='録画')
speed_high_slider = ipywidgets.IntSlider(description='High', min=0, max=100, step=1, value=20, orientation='horizontal')
speed_low_slider = ipywidgets.IntSlider(description='Low', min=0, max=100, step=1, value=10, orientation='horizontal')
steering_gain_slider = ipywidgets.FloatSlider(description='Steering gain', min=0.1, max=2.0, step=0.1, value=1.0, orientation='horizontal')

record = False
running = False
load = False

########################################
# remap
########################################
def remap(x, oMin, oMax, nMin, nMax):
    """
    args:
      x: input value
      oMin: old minimum value
      oMax: old maximum value
      nMin: new minimum value
      nMax: new maximum value
    """

    #range check
    if oMin == oMax:
        print("Warning: Zero input range")
        return None

    if nMin == nMax:
        print("Warning: Zero output range")
        return None

    #check reversed input range
    reverseInput = False
    oldMin = min( oMin, oMax )
    oldMax = max( oMin, oMax )
    if not oldMin == oMin:
        reverseInput = True

    #check reversed output range
    reverseOutput = False   
    newMin = min( nMin, nMax )
    newMax = max( nMin, nMax )
    if not newMin == nMin :
        reverseOutput = True

    portion = (x-oldMin)*(newMax-newMin)/(oldMax-oldMin)
    if reverseInput:
        portion = (oldMax-x)*(newMax-newMin)/(oldMax-oldMin)

    result = portion + newMin
    if reverseOutput:
        result = newMax - portion

    return result

def handle(x):
    global pwm_right,pwm_left,STEERING_CH,PCA9685
    x = remap(x, 224, 0, pwm_right, pwm_left)
    PCA9685.set_channel_value(STEERING_CH, x)
    
def live():
    global running,camera,IMG_WIDTH,record,model_trt,preprocess,pwm_stop,save_dir,num,cv2,count,speed_high_slider,speed_low_slider,steering_gain_slider
    
    count = 1
    num = 1
    
    while running:
        image = camera.read()
        if record == True:
            remarked_img = image.copy()
        image = preprocess(image).half()
        output = model_trt(image).detach().cpu().numpy().flatten()
        x = float(output[0]) * steering_gain_slider.value
        y = float(output[1])
        speed = float(output[3])
        x = int(IMG_WIDTH * (x / 2.0 + 0.5))
        y = int(IMG_WIDTH * (y / 2.0 + 0.5))
        speed = int(IMG_WIDTH * (speed / 2.0 + 0.5))
        handle(x)
        
        if direction == 0:
            pwm_speed_high = pwm_stop + speed_high_slider.value
            pwm_speed_low = pwm_stop + speed_low_slider.value
        else:
            pwm_speed_high = pwm_stop - speed_high_slider.value
            pwm_speed_low = pwm_stop - speed_low_slider.value
        #PCA9685.set_channel_value(THROTTLE_CH, pwm_speed)
        
        if speed > 112:
            PCA9685.set_channel_value(THROTTLE_CH, pwm_speed_high)
        elif speed <= 112:
            PCA9685.set_channel_value(THROTTLE_CH, pwm_speed_low)

        if record == True:
            if num % 2 == 0:
                name = "0_0_{:0=5}.jpg".format(num)
                image_path = os.path.join(save_dir, name)
                cv2.imwrite(image_path, remarked_img)
            num += 1
        count += 1
        
        #write_log("Count:" + str(count))
        
def run(change):
    global running,execute_thread,name_widget,save_dir,start_time,load,speed_high_slider,speed_low_slider,steering_gain_slider
    
    if load == False:
        write_log("モデルが読み込まれていません")
        return

    if running == False:
        if record == True:
            if name_widget.value != "":
                save_dir = "run/" + name_widget.value + "/xy/" 
                if not os.path.exists(save_dir):
                    subprocess.call(['mkdir', '-p', save_dir])
                write_log(save_dir + "にデータを保存します。")            

                running = True
                execute_thread = threading.Thread(target=live)
                execute_thread.start()
                start_time = time.time()
                write_log("AIが起動しました。")
                if direction == 0:
                    pwm_speed_high = pwm_stop + speed_high_slider.value
                    pwm_speed_low = pwm_stop + speed_low_slider.value
                else:
                    pwm_speed_high = pwm_stop - speed_high_slider.value
                    pwm_speed_low = pwm_stop - speed_low_slider.value
                write_log("Steering Gain:" + str(steering_gain_slider.value) + " Speed High:" + str(pwm_speed_high) + " Low:" + str(pwm_speed_low) + "で走行開始します。")
            else:
                write_log("映像の保存先を入力してください。")
        else:
            running = True
            execute_thread = threading.Thread(target=live)
            execute_thread.start()
            start_time = time.time()
            write_log("AIが起動しました。") 
            if direction == 0:
                pwm_speed_high = pwm_stop + speed_high_slider.value
                pwm_speed_low = pwm_stop + speed_low_slider.value
            else:
                pwm_speed_high = pwm_stop - speed_high_slider.value
                pwm_speed_low = pwm_stop - speed_low_slider.value
            write_log("Steering Gain:" + str(steering_gain_slider.value) + " Speed High:" + str(pwm_speed_high) + " Low:" + str(pwm_speed_low) + "で走行開始します。")
        
    
def stop(change):
    global running,execute_thread,end_time,start_time,count,pwm_stop
    if running == True:
        PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
        try:
            end_time = time.time() - start_time
            fps = count/int(end_time)
            process_time = int((end_time/count)*1000)
        except:
            fps = -1
            process_time = -1
        
        write_log("AIを停止しました。")
        write_log("処理結果:FPS: " + str(round(fps,2)) + ",処理回数: " + str(count) + ",　処理時間(1回平均値): " + str(process_time) + " ms")
        running = False
        execute_thread.join()
    else:
        PCA9685.set_channel_value(THROTTLE_CH, pwm_stop)
        write_log("現在AIは動いていません。")

def load_model(change):
    global model_trt,model_widget,load
    try:
        write_log(model_widget.value + "の読込を実行します(初回は時間がかかります)。")
        model_trt = TRTModule()
        model_trt.load_state_dict(torch.load(model_widget.value))
        write_log(model_widget.value + "の読込に成功しました。")
        load = True
    except:
        write_log("[Error]" + model_widget.value + "の読込に失敗しました。")
        
def model_list(change):
    global model_widget
    try:
        files = glob.glob('./model_trt/*.pth', recursive=True)
        model_widget.options = files
        ts = os.path.getctime(files[0])
        d = datetime.datetime.fromtimestamp(ts)
        s = d.strftime('%Y-%m-%d %H:%M:%S')
        model_time_widget.value = s
    except:
        model_widget.options = []
model_list("list")

def change_file(change):
    file = model_widget.value
    ts = os.path.getctime(file)
    d = datetime.datetime.fromtimestamp(ts)
    s = d.strftime('%Y-%m-%d %H:%M:%S')
    model_time_widget.value = s
model_widget.observe(change_file, names='value')

def on_video(change):
    global record
    record^=True
    
load_button.on_click(load_model)
run_button.on_click(run)
stop_button.on_click(stop)
record_box.observe(on_video)

走行までの流れは以下の通りです。

1. <b>学習済みモデルを指定してLoadする</b><br>
2. <b>Steering gainの値を調整する</b><br>
初期値は1.0です。0.1〜2.0の値で設定できる、1.0より大きな値にするとハンドルのキレ角が鋭くなる<br>
3. <b>High, Lowの値を調整する</b><br>
4. <b>[オプション] 走行動画を録画する場合は、録画にチェックマークをいれて、保存ファイル名を指定する</b><br>
./runフォルダに保存<br>
5. <b>runボタンを押して、プロポの裏側のボタンを押して AIモードで自動走行開始する</b><br>
6. <b>終了時は、stopボタンを押す</b><br>
録画のチェックマークがついている場合は、stopで録画も終了<br>
<br>
カメラが60fpsで動いている場合は16ms以内、カメラが30fpsで動いている場合は、33ms以内での処理完了が正常な挙動となります。

In [ ]:
data_collection_widget = ipywidgets.VBox([
    ipywidgets.HBox([model_widget,model_time_widget,load_button]),
    ipywidgets.HBox([steering_gain_slider, speed_high_slider, speed_low_slider]),
    ipywidgets.HBox([record_box,name_widget,run_button, stop_button]),
    process_widget
])
display(data_collection_widget)

### カメラの終了処理(必須)

In [ ]:
import time

camera.running = False
camera.cap.release()